In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.interpolate import interp1d
import numpy as np

matplotlib.use('TkAgg')
plt.ion()  # enable interactive mode for TkAgg backend

SKIP_CELL = False  # Change to False to allow execution

In [2]:
def select_directory(root, close=True):
    """
    Opens a directory selection dialog and returns the selected path
    
    Args:
        root: Tkinter root window
        close: Close the root window after selection (default: True)
        
    Returns:
        str: Selected directory path or empty string if cancelled
    """
    directory = filedialog.askdirectory(
        parent=root,
        initialdir=".",
        title="Select Directory"
    )
    if close:
        root.destroy()
    return directory

def select_files(root, close=True):
    """
    Opens a file selection dialog and returns the selected paths
    
    Args:
        root: Tkinter root window
        close: Close the root window after selection (default: True)
        
    Returns:
        list: Selected file paths or empty list if cancelled
    """
    files = filedialog.askopenfilenames(
        parent=root,
        initialdir=".",
        title="Select Files",
        filetypes=(("All files", "*.*"),)
    )
    if close:
        root.destroy()
    return files

def getdir():
    root = tk.Tk()
    return select_directory(root, close=True)

def getfiles():
    root = tk.Tk()
    return select_files(root, close=True)


In [3]:
if SKIP_CELL == False:
    d = getdir() # open dir # run this cell to open a dir and save it on the variable d
    print(d)

In [4]:
if SKIP_CELL == False:
    f = getfiles() # open files # run this cell to open files and save them to the array f
    print(len(f), f)